## Import Packages

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Load Data

In [13]:
x_test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
y_train = train['label']
x_train = train.drop(['label'],axis=1)
# free some space
del train

### Normalization

In [14]:
x_train = x_train/255.0
x_test = x_test/255.0

### Reshape

In [15]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = x_train.values.reshape(-1,28,28,1)
X_test = x_test.values.reshape(-1,28,28,1)

### Label Encoding

In [16]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
Y_train = to_categorical(y_train, num_classes=10)

### Split into training and validation set

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

## Create Neural Network Architecture 

In [19]:
input_shape = (28, 28, 1)
# Creating a Sequential Model and adding the layers
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [20]:
from keras.optimizers import RMSprop
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

#compile model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

## Set up Early Stopping

In [21]:
# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='loss', patience=1),
             ModelCheckpoint(filepath='best_model.h5', monitor='loss', save_best_only=True)]

## Train Neural Network

In [24]:
history = model.fit(X_train, Y_train, batch_size = 86, epochs = 50, 
          validation_data = (X_val, Y_val), callbacks=callbacks)

Train on 37800 samples, validate on 4200 samples
Epoch 1/50
37800/37800 [==============================] - 1285s 34ms/step - loss: 0.1140 - acc: 0.9651 - val_loss: 0.0511 - val_acc: 0.9826
Epoch 2/50
 6192/37800 [===>..........................] - ETA: 26:13 - loss: 0.0816 - acc: 0.9774

KeyboardInterrupt: 

## Training History Visualization

In [ ]:

# Plot accuracy & loss values
plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(x_test)

results = predictions.argmax(axis=1)
results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("solution.csv",index=False)

In [ ]:
model.summary()